In [12]:
import os
folder_path = "/Users/akshararamprasad/Documents/Fashion/imagesdata"
file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
print(len(file_names))

44441


In [13]:
import torch
print(torch.__version__)

2.1.2


In [14]:
print(type(file_names[0]))

<class 'str'>


In [15]:
import pandas as pd
import numpy as np
df = pd.read_csv("/Users/akshararamprasad/Documents/Fashion/styles.csv", on_bad_lines="skip")
current = []
for x in df.id.values:
    if str(x)+".jpg" not in file_names:
        current.append(False)
    else:
        current.append(True)
df["Good"] = np.array(current)
df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,Good
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,True
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,True
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,True
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,True
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,True


In [16]:
df["Good"].value_counts()

Good
True     44419
False        5
Name: count, dtype: int64

In [17]:
df["baseColour"].unique()

array(['Navy Blue', 'Blue', 'Silver', 'Black', 'Grey', 'Green', 'Purple',
       'White', 'Beige', 'Brown', 'Bronze', 'Teal', 'Copper', 'Pink',
       'Off White', 'Maroon', 'Red', 'Khaki', 'Orange', 'Coffee Brown',
       'Yellow', 'Charcoal', 'Gold', 'Steel', 'Tan', 'Multi', 'Magenta',
       'Lavender', 'Sea Green', 'Cream', 'Peach', 'Olive', 'Skin',
       'Burgundy', 'Grey Melange', 'Rust', 'Rose', 'Lime Green', 'Mauve',
       'Turquoise Blue', 'Metallic', 'Mustard', 'Taupe', 'Nude',
       'Mushroom Brown', nan, 'Fluorescent Green'], dtype=object)

In [19]:
idx = []
category = []
color = []
for a, b, c, d in zip(df.id.values, df.masterCategory.values, df.baseColour.values, df.Good.values):
    if d:
        idx.append(a)
        category.append(b)
        color.append(c)

In [20]:
data = pd.DataFrame()
data["id"] = np.array(idx)
data["masterCategory"] = np.array(category)
data["baseColour"] = np.array(color)

In [21]:
data.shape

(44419, 3)

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim 
import torchvision.transforms as transforms
import torchvision
import os
import pandas as pd
from PIL import Image
from torch.utils.data import (
    Dataset,
    DataLoader,
)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to the same size
    transforms.ToTensor(),
])

class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = data
        self.class_mapping = {color: idx for idx, color in enumerate(self.annotations["baseColour"].unique())}
        self.annotations["baseColour"] = self.annotations["baseColour"].map(self.class_mapping)
        self.root_dir = root_dir
        self.transform = transform


    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, str(self.annotations.iloc[index, 0]) + ".jpg")
        image = Image.open(img_path).convert("RGB")
        y_label = torch.tensor(int(self.annotations.iloc[index, 2]))

        if self.transform:
            image = self.transform(image)

    # Rearrange dimensions: (batch_size, height, width, channels) to (batch_size, channels, height, width)
        # image = image.permute(2, 0, 1)

        return image, y_label

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
in_channel = 3
num_classes = 7
learning_rate = 3e-4
batch_size = 32
num_epochs = 5

# Load Data
dataset = CustomDataset(
    csv_file="styles.csv",
    root_dir="/Users/akshararamprasad/Documents/Fashion/imagesdata",
    transform=transform,
)

train_set, test_set = torch.utils.data.random_split(dataset, [35000, 9419])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

# Model
model = torchvision.models.googlenet(weights="DEFAULT")

for param in model.parameters():
    param.requires_grad = True

# final layer is not frozen
model.fc = nn.Linear(in_features=1024, out_features=len(data['baseColour'].unique()))
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

In [23]:
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()

print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

KeyboardInterrupt: 

In [ ]:
torch.save(model, 'googlenet_colormodel.pth')

In [ ]:
# predictions
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
from PIL import Image
import torchvision.models as models

loaded_model = torch.load("googlenet_colormodel.pth")
loaded_model.eval()

# Define the image preprocessing function
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = transform(image)
    input_batch = input_tensor.unsqueeze(0)  # Add batch dimension
    return input_batch

# Define the prediction function
def predict_image(model, image_path):
    input_tensor = preprocess_image(image_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_tensor = input_tensor.to(device)
    model = model.to(device)
    with torch.no_grad():
        output = model(input_tensor)

    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    return probabilities

# Specify the path to the new image
new_image_path = '/Users/akshararamprasad/Documents/Fashion/imagesdata/34123.jpg'

# Make predictions for the new image
probabilities = predict_image(loaded_model, new_image_path)

# Display the predicted probabilities for each class
print(probabilities)